In [1]:
import torch
import numpy as np
from torch.utils.cpp_extension import load_inline
from torch.profiler import profile, record_function, ProfilerActivity

In [2]:
def trace_handler(prof):
    print(prof.key_averages().table(sort_by="self_cuda_time_total", row_limit=-1))
    prof.export_chrome_trace("tmp/test_trace_" + str(prof.step_num) + ".json")

def profile_func(func, *tensors, trace_handler=trace_handler):
        
    """ In this example with wait=1, warmup=1, active=2, repeat=1, profiler will skip the first step/iteration,
        start warming up on the second, record the third and the forth iterations, after which the trace will become available
        and on_trace_ready (when set) is called; the cycle repeats starting with the next step """
    with torch.profiler.profile(
        activities=[
            torch.profiler.ProfilerActivity.CPU,
            torch.profiler.ProfilerActivity.CUDA,
        ],
        schedule=torch.profiler.schedule(wait=1, warmup=1, active=2, repeat=1), on_trace_ready=trace_handler
        # on_trace_ready=torch.profiler.tensorboard_trace_handler('./log')
        # used when outputting for tensorboard
        ) as p:
            for iter in range(10):
                func(*tensors)
                # send a signal to the profiler that the next iteration has started
                p.step()

### Simple paralellel histogram with atomic operations

In [36]:
cuda_source = '''
#include <stdio.h>
#include <torch/extension.h>
#include <c10/cuda/CUDAException.h>
#include <cuda.h>
#include <ATen/cuda/Atomic.cuh>`

__global__ void histo_kernel(const int* data, int length, int* histo) {
    int i = blockIdx.x * blockDim.x + threadIdx.x;
    // Ensure `i` is within bounds before accessing data and histo
    if (i < length) {
        int alphabet_position = data[i] - 97; // Python unicode system for lowercase letters is [97,122]
        // Validate alphabet_position to be within the valid range
        if (0 <= alphabet_position && alphabet_position <= 26) { 
            // Use atomicAdd for thread-safe histogram updates
            gpuAtomicAdd(&histo[alphabet_position], 1);
        }
    }
}

torch::Tensor parallel_histo(torch::Tensor data) {

    auto options = torch::TensorOptions().device(torch::kCUDA, 0).dtype(torch::kInt32);

    auto length = data.size(0);
    auto histo = torch::zeros({26}, options);

    // Launch the kernel with necessary configuration
    int num_blocks = (length + 127) / 128; // Dynamic block allocation
    histo_kernel<<<num_blocks, 128>>>(data.data_ptr<int>(), length, histo.data_ptr<int>());
    return histo;
}
'''

cpp_source = "torch::Tensor parallel_histo(torch::Tensor data);"

In [37]:
parallel_histo_extension = load_inline(
    name='parallel_histo_extension',
    cpp_sources=cpp_source,
    cuda_sources=cuda_source,
    functions=['parallel_histo'],
    with_cuda=True,
    extra_cuda_cflags=["-O2"],
    build_directory='tmp',
)


In [62]:
data = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Odio ut sem nulla pharetra diam sit amet nisl. Sed risus pretium quam vulputate dignissim. Sapien pellentesque habitant morbi tristique senectus. Varius sit amet mattis vulputate enim nulla aliquet. Tristique et egestas quis ipsum suspendisse ultrices gravida dictum fusce. Blandit cursus risus at ultrices mi. Sem fringilla ut morbi tincidunt. Sit amet nisl purus in mollis. Neque viverra justo nec ultrices. In hendrerit gravida rutrum quisque non tellus. Nulla porttitor massa id neque. Et egestas quis ipsum suspendisse ultrices gravida dictum fusce. Ultrices gravida dictum fusce ut placerat orci nulla pellentesque dignissim.'
t_data = torch.tensor([x for x in data.encode('utf-8')], device='cuda', dtype=torch.int)

In [63]:
parallel_histo_extension.parallel_histo(t_data)

tensor([44,  4, 21, 26, 64,  4, 11,  3, 71,  1,  0, 35, 29, 31, 19, 17, 13, 38,
        60, 59, 57,  8,  0,  0,  0,  0], device='cuda:0', dtype=torch.int32)

In [64]:
%%timeit
parallel_histo_extension.parallel_histo(t_data)

8.55 µs ± 71.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Check results with numpy

In [59]:
arr = t_data.detach().cpu().numpy()
letters_arr = np.array([x - 97 for x in arr if x >= 97])

unique, counts = np.unique(letters_arr, return_counts=True)
np_hist = np.zeros((26,), dtype='int')
for idx, elem in zip(unique,counts):
    np_hist[idx] = elem

print(np_hist)

[2 1 1 1 2 0 1 2 5 0 0 1 1 2 1 0 0 2 3 2 0 0 0 0 0 0]


### Parallel histogram with Privatization in shared memory (contiguous partitioning)

![](images/contigous.png)

In [6]:
cuda_source = '''
#include <stdio.h>
#include <torch/extension.h>
#include <c10/cuda/CUDAException.h>
#include <cuda.h>
#include <ATen/cuda/Atomic.cuh>`

#define HIST_LENGTH 26
#define NUM_BINS 26
#define CFACTOR 3

__global__ void histo_kernel(const int* data, int length, int* histo) {

    // Initialize private bins. 
    __shared__ unsigned int histo_s[HIST_LENGTH];
    
    for(unsigned int binIdx=threadIdx.x; binIdx<NUM_BINS; binIdx+= blockDim.x) {
    // Every thread will fill 1 position until all positions in the shared array are filled with zeros
        histo_s[binIdx] = 0u;
    }
    __syncthreads(); // Wait for all threads to finish the initialization
    
    // Histogram
    unsigned int tid = blockIdx.x*blockDim.x + threadIdx.x;
    for(unsigned int i=tid*CFACTOR; i<min((tid+1)*CFACTOR,length); ++i) {
        int alphabet_position = data[i] - 97; // Python unicode system for lowercase letters is [97,122]
        // Validate alphabet_position to be within the valid range
        if (0 <= alphabet_position && alphabet_position <= 26) { 
            // Use atomicAdd for thread-safe histogram updates
            atomicAdd(&histo_s[alphabet_position], 1);
        }
    }
    __syncthreads();
    // Commit to global memory
    for(unsigned int binIdx=threadIdx.x; binIdx<NUM_BINS; binIdx+=blockDim.x){
        unsigned int binValue = histo_s[binIdx];
        if(binValue > 0) { 
            atomicAdd(&(histo[binIdx]), binValue);
        }
    }
}

torch::Tensor parallel_histo(torch::Tensor data) {

    auto options = torch::TensorOptions().device(torch::kCUDA, 0).dtype(torch::kInt32);

    auto length = data.size(0);
    auto histo = torch::zeros({26}, options);

    // Launch the kernel with necessary configuration
    // We define 3 blocks each of which will have a private copy
    histo_kernel<<<3, 128>>>(data.data_ptr<int>(), length, histo.data_ptr<int>());
    return histo;
}
'''

cpp_source = "torch::Tensor parallel_histo(torch::Tensor data);"

In [7]:
data = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Odio ut sem nulla pharetra diam sit amet nisl. Sed risus pretium quam vulputate dignissim. Sapien pellentesque habitant morbi tristique senectus. Varius sit amet mattis vulputate enim nulla aliquet. Tristique et egestas quis ipsum suspendisse ultrices gravida dictum fusce. Blandit cursus risus at ultrices mi. Sem fringilla ut morbi tincidunt. Sit amet nisl purus in mollis. Neque viverra justo nec ultrices. In hendrerit gravida rutrum quisque non tellus. Nulla porttitor massa id neque. Et egestas quis ipsum suspendisse ultrices gravida dictum fusce. Ultrices gravida dictum fusce ut placerat orci nulla pellentesque dignissim."
t_data = torch.tensor([x for x in data.encode('utf-8')], device='cuda', dtype=torch.int)

In [8]:
parallel_histo_extension = load_inline(
    name='parallel_histo_extension',
    cpp_sources=cpp_source,
    cuda_sources=cuda_source,
    functions=['parallel_histo'],
    with_cuda=True,
    extra_cuda_cflags=["-O2"],
    build_directory='tmp',
)


In [9]:
parallel_histo_extension.parallel_histo(t_data)

tensor([44,  4, 21, 26, 64,  4, 11,  3, 71,  1,  0, 35, 29, 31, 19, 17, 13, 38,
        60, 59, 57,  8,  0,  0,  0,  0], device='cuda:0', dtype=torch.int32)

In [10]:
%%timeit
parallel_histo_extension.parallel_histo(t_data)

8.23 µs ± 39.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


### Parallel histogram with Privatization in shared memory (interleaved partitioning)

![](images/interleaved.png)

In [12]:
cuda_source = '''
#include <stdio.h>
#include <torch/extension.h>
#include <c10/cuda/CUDAException.h>
#include <cuda.h>
#include <ATen/cuda/Atomic.cuh>`

#define HIST_LENGTH 26
#define NUM_BINS 26
#define CFACTOR 3

__global__ void histo_kernel(const int* data, int length, int* histo) {

    // Initialize private bins. 
    __shared__ unsigned int histo_s[HIST_LENGTH];
    
    for(unsigned int binIdx=threadIdx.x; binIdx<NUM_BINS; binIdx+= blockDim.x) {
    // Every thread will fill 1 position until all positions in the shared array are filled with zeros
        histo_s[binIdx] = 0u;
    }
    __syncthreads(); // Wait for all threads to finish the initialization
    
    // Histogram
    unsigned int tid = blockIdx.x*blockDim.x + threadIdx.x;
    for(unsigned int i= tid; i<length; i += blockDim.x*gridDim.x) {
        int alphabet_position = data[i] - 97; // Python unicode system for lowercase letters is [97,122]
        // Validate alphabet_position to be within the valid range
        if (0 <= alphabet_position && alphabet_position <= 26) { 
            // Use atomicAdd for thread-safe histogram updates
            atomicAdd(&histo_s[alphabet_position], 1);
        }
    }
    __syncthreads();
    // Commit to global memory
    for(unsigned int binIdx=threadIdx.x; binIdx<NUM_BINS; binIdx+=blockDim.x){
        unsigned int binValue = histo_s[binIdx];
        if(binValue > 0) { 
            atomicAdd(&(histo[binIdx]), binValue);
        }
    }
}

torch::Tensor parallel_histo(torch::Tensor data) {

    auto options = torch::TensorOptions().device(torch::kCUDA, 0).dtype(torch::kInt32);

    auto length = data.size(0);
    auto histo = torch::zeros({26}, options);

    // Launch the kernel with necessary configuration
    // We define 3 blocks each of which will have a private copy
    histo_kernel<<<3, 128>>>(data.data_ptr<int>(), length, histo.data_ptr<int>());
    return histo;
}
'''

cpp_source = "torch::Tensor parallel_histo(torch::Tensor data);"

In [13]:
parallel_histo_extension = load_inline(
    name='parallel_histo_extension',
    cpp_sources=cpp_source,
    cuda_sources=cuda_source,
    functions=['parallel_histo'],
    with_cuda=True,
    extra_cuda_cflags=["-O2"],
    build_directory='tmp',
)


In [14]:
parallel_histo_extension.parallel_histo(t_data)

tensor([44,  4, 21, 26, 64,  4, 11,  3, 71,  1,  0, 35, 29, 31, 19, 17, 13, 38,
        60, 59, 57,  8,  0,  0,  0,  0], device='cuda:0', dtype=torch.int32)

In [16]:
%%timeit
parallel_histo_extension.parallel_histo(t_data)

9.24 µs ± 264 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
